## Step 1: Pre-Preprocessing

In [1]:
%load_ext autoreload
%autoreload 2

# import preprocessing code
from src.preprocess import PreProcessor, df_to_train_set, df_to_XY

Using TensorFlow backend.


In [3]:
# save paths to the available datasets
from typing import NamedTuple, List

class Dataset(NamedTuple):
    """
    Interface for accessing data folders.
    """
    title: str
    preprocessed_folder: str
    raw_folders: List[str]

SAMPLE_DATA = Dataset(
    title = "sample_data",
    preprocessed_folder = "../de-ID_data/preprocessed/sample_data/",
    raw_folders = ["docs/Track1-de-indentification/PHI/"]
)

GOLD_1 = Dataset(
    title = "gold_1",
    preprocessed_folder = "../de-ID_data/preprocessed/gold_1/",
    raw_folders = ["../de-ID_data/raw/training-PHI-Gold-Set1/"]
)

GOLD_FULL = Dataset(
    title = "gold_full",
    preprocessed_folder = "../de-ID_data/preprocessed/gold_full/",
    raw_folders = ["../de-ID_data/raw/training-PHI-Gold-Set1/","../data/raw/training-PHI-Gold-Set2/"]
)

GOLD_TEST = Dataset(
    title = "gold_test",
    preprocessed_folder = "../de-ID_data/preprocessed/gold_test/",
    raw_folders = ["../de-ID_data/raw/testing-PHI-Gold-fixed/"]
)

DATASETS = [SAMPLE_DATA,GOLD_1,GOLD_FULL, GOLD_TEST]

## Step 2: Preprocessing

In [4]:
# pick dataset and define loading boolean
train_data = DATASETS[2]
# train_data = DATASETS[0]
test_data = DATASETS[3]
isLoading = True

In [5]:
# attach data to PreProcessor object.
pp = PreProcessor(train_data.title)
if isLoading:
    X_train,y_train,X_train_words,df_train = pp.get_data(train_data.preprocessed_folder,isLoading = isLoading)
else:
    X_train,y_train,X_train_words,df_train = pp.get_data(train_data.raw_folders,isLoading = isLoading)
print("max length: ",pp.max_len)

Loading preprocessed data...
100%|██████████| 31535/31535 [00:03<00:00, 8888.65it/s]
Preprocessing complete.
max length:  1567


In [ ]:
# data exploration
df_train.head()

In [ ]:
# load test set
isLoadingTest = True
if isLoadingTest:
    X_test,y_test,X_test_words,df_test = pp.create_test_set(test_data.preprocessed_folder,isLoadingTest,test_data.title)
else:
    X_test,y_test,X_test_words,df_test = pp.create_test_set(test_data.raw_folders,isLoadingTest,test_data.title)

In [ ]:
# test data exploration
df_test.head()

In [6]:
# import model stuff
from src.models.baseline import BaselineModel
from src.models.bilstm import BiLSTM
from src.models.bilstm_crf import BiLSTM_CRF
from src.models.transformer import Transformer
from src.models.transformer_crf import Transformer_CRF
from src.models.transformer_bilstm import TransformerBiLSTM
from src.models.bilstm_chars import BiLSTM_Chars
from src.models.bilstm_chars_crf import BiLSTM_Chars_CRF
from pipeline.visualization import sample_output
from pipeline.train import train
from random import randint
from sklearn.utils import shuffle
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from src.converter import get_label_positions, bio_to_i2d2
import xml.etree.ElementTree as ET
from typing import NamedTuple, List

In [ ]:
# check if GPU is available
assert tf.test.is_built_with_cuda()
physical_devices = tf.config.list_physical_devices('GPU') 
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num GPUs:", len(physical_devices)) 
tf.compat.v1.RunOptions(report_tensor_allocations_upon_oom = True)

In [7]:
# build model
# model = BaselineModel(pp.vocab_size,pp.tag_size,pp.max_len)
# model = BiLSTM(pp.vocab_size,pp.tag_size,pp.max_len)
model = BiLSTM_CRF(pp.vocab_size,pp.tag_size,pp.max_len)
# model = Transformer(pp.vocab_size,pp.tag_size,pp.max_len)
# model = Transformer_CRF(pp.vocab_size, pp.tag_size, pp.max_len)
# model = TransformerBiLSTM(pp.vocab_size, pp.tag_size, pp.max_len)
# model = BiLSTM_Chars(pp.vocab_size, pp.tag_size, pp.max_len,pp.idx2word)
# model = BiLSTM_Chars_CRF(pp.vocab_size, pp.tag_size, pp.max_len,pp.idx2word)

Num GPUs Available:  1


In [8]:
# configure checkpoints and checkpoint manager
checkpoint_dir = 'models/checkpoints/' + train_data.title + '/' + model.title + '/' 
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
checkpoint = tf.train.Checkpoint(model=model)
manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir, max_to_keep=10)

In [9]:
# restore checkpoint
checkpoint.restore(manager.latest_checkpoint)
if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))

Restored from models/checkpoints/gold_full/bi-lstm-crf/ckpt-10


In [ ]:
# train
print("Training ",model.title)
losses = train(model,X_train,y_train,X_train_words,batch_size = 32, epochs=10, lr = 0.0005, sample_interval=10, manager=manager, pp=pp)

In [ ]:
# sample a random train output
sample_output(model,X_train,y_train, pp = pp,rand_idx=None)

In [ ]:
# sample a random test output
# print(X_test_words[0][:30])
sample_output(model,X_test,y_test, pp = pp,rand_idx=None, words = None)

In [13]:
# test model
from pipeline.test import test_to_i2d2

covid_path = ["../de-ID_data/covid/"]

X_test,y_test,X_test_words,df_test = pp.create_test_set(covid_path,False,"covid")

100%|██████████| 60/60 [00:00<00:00, 4168.74it/s]Preprocessing data...
# of Tag Processing Errors:  0
Files with errors:  []



In [14]:
# df_test.head(2000)

# sample_output(model,X_test,y_test, pp = pp,rand_idx=None, words = None)

test_to_i2d2(model,df_test, pp, checkpoint, manager)

c:\Users\abdul\Desktop\de-identification
Loading checkpoint...
100%|██████████| 3/3 [00:16<00:00,  5.33s/it]
